In [344]:
from cgi import test
import re
import os
import glob
import random
from nltk.corpus import stopwords
import nltk
import string
from pprint import pprint
from utils import *
from collections import Counter
import math

In [345]:
positrain, negatrain, voc = load_training_set(0.01,0.01)

In [346]:
posinum = len(positrain)
neganum = len(negatrain)
suminstance = neganum+posinum
print(len(negatrain))
print(len(positrain))
print(len(voc))

134
136
8337


In [347]:
sum([len(i) for i in positrain])

16906

In [348]:
countdict =  [dict(Counter(i)) for i in positrain]
testinstance = countdict[11]

In [349]:
def train(positivedata, negativedata):
    posidict = dict(Counter(sum(positivedata,[])))
    print(len(sum(positivedata,[])))
    print(len(sum(negativedata,[])))
    negadict = dict(Counter(sum(negativedata,[])))  
    return posidict, negadict

posidict, negadict = train(positrain, negatrain)

16906
14385


In [356]:

def probilityof(positivetrain, negativetrain, positivedict, negativedict, category: string, instance, log: bool, laplacesmooth: bool, smoothconstant = 1):

    posinum = len(positivetrain)
    neganum = len(negativetrain)
    suminstance = neganum+posinum   

    if category == "posi" or category == "positive":
        num = posinum
        trainset = positivetrain
        dictuse = positivedict
    else:
        num = neganum
        trainset = negativetrain
        dictuse = negativedict

    plist = []
    p0 = num/suminstance
    plist.append(p0)
    denominator = sum([len(i) for i in trainset])
    # print(sum(dictuse.values())-denominator)
    vsize = len(dictuse)

    for i in instance:
        if (not laplacesmooth):
            if i in dictuse:
                probinstance = (dictuse[i]/denominator)
            else: 
                continue
        else: # Smooth
            if i in dictuse:
                probinstance = ((dictuse[i]+smoothconstant)/(denominator+smoothconstant*vsize))
            else: 
                probinstance = ((smoothconstant)/(smoothconstant*vsize))
        plist.append(probinstance)

    if log:
        loglist = list(map(math.log10, plist))
        return sum(loglist)
    
    # if 0 not in plist:
    #     print("prob is not zero")

    return math.prod(plist)

In [358]:
pposi = probilityof(positrain,negatrain,posidict,negadict,'posi',testinstance,True,False)
pnega = probilityof(positrain,negatrain,posidict,negadict,'nega',testinstance,True,False)

print(pposi)
print(pnega)

老8
-130.42205168627697
-86.75026405187567
